# MSR Image Recognition Challenge (IRC) @ACM Multimedia 2016

Using Resnet-18

Author: YinTaiChen

![title](https://www.microsoft.com/en-us/research/wp-content/uploads/2014/03/irc_ms-celeb-1m-sample0.jpg)

## Import everything

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os

## Define Dataset Loader

In [2]:
import torch.utils.data as data

from PIL import Image
import os
import os.path

def default_loader(path):
	return Image.open(path).convert('RGB')

def default_flist_reader(flist):
	"""
	flist format: impath label\nimpath label\n ...(same to caffe's filelist)
	"""
	imlist = []
	with open(flist, 'r') as rf:
		for line in rf.readlines():
			impath, imlabel = line.strip().split()
			imlist.append( (impath, int(imlabel)) )
					
	return imlist

class ImageFilelist(data.Dataset):
	def __init__(self, root, flist, transform=None, target_transform=None,
			flist_reader=default_flist_reader, loader=default_loader):
		self.root   = root
		self.imlist = flist_reader(flist)		
		self.transform = transform
		self.target_transform = target_transform
		self.loader = loader

	def __getitem__(self, index):
		impath, target = self.imlist[index]
		img = self.loader(os.path.join(self.root,impath))
		if self.transform is not None:
			img = self.transform(img)
		if self.target_transform is not None:
			target = self.target_transform(target)
		
		return img, target

	def __len__(self):
		return len(self.imlist)


## Split dataset into training set and testing set

## Download pretrained Resnet

In [3]:
model = models.resnet18(pretrained=True)

## Final fc layer replacement

In [4]:
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 79077)

## Use CUDA if available

In [5]:
use_gpu = torch.cuda.is_available()

if use_gpu:
    model = model.cuda()

## Loss function, SGD and learning rate scheduler

In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

## Image pre-processing and transformed to tensor

In [7]:
data_transforms = transforms.Compose([
    transforms.RandomSizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

## Load training set

In [8]:
trainset = ImageFilelist(root="./msr", flist="train_data.txt", transform=data_transforms)

## Load testing set

In [9]:
testset = ImageFilelist(root="./msr", flist="test_data.txt", transform=data_transforms)

## Two phases to iterate through

In [10]:
image_datasets = {
    'train': trainset,
    'test': testset
}

## Size of dataset (for precision)

In [14]:
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'test']}

## Make datasets iterable

In [11]:
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'test']}

## Define function for model training and testing 

In [12]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = model.state_dict()
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and testing phase
        for phase in ['train', 'test']:
            if phase == 'train':
                scheduler.step()
                model.train(True)  # Set model to training mode
            else:
                model.train(False)  # Set model to testing mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for data in dataloaders[phase]:
                # get the inputs
                inputs, labels = data

                # wrap them in Variable
                if use_gpu:
                    inputs = Variable(inputs.cuda())
                    labels = Variable(labels.cuda())
                else:
                    inputs, labels = Variable(inputs), Variable(labels)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                outputs = model(inputs)
                _, preds = torch.max(outputs.data, 1)
                loss = criterion(outputs, labels)

                # backward + optimize only if in training phase
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

                # statistics
                running_loss += loss.data[0]
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'test' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = model.state_dict()

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best test Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

## Train and Test

In [15]:
trained_model = train_model(
    model = model,
    criterion = criterion,
    optimizer = optimizer,
    scheduler = exp_lr_scheduler
    )

Epoch 0/24
----------
train Loss: 2.0536 Acc: 0.0181
test Loss: 1.7424 Acc: 0.0596

Epoch 1/24
----------
train Loss: 1.5168 Acc: 0.1310
test Loss: 1.2277 Acc: 0.2478

Epoch 2/24
----------
train Loss: 1.0649 Acc: 0.3245
test Loss: 0.8792 Acc: 0.4294

Epoch 3/24
----------
train Loss: 0.8010 Acc: 0.4705
test Loss: 0.7130 Acc: 0.5273

Epoch 4/24
----------
train Loss: 0.6492 Acc: 0.5625
test Loss: 0.6101 Acc: 0.5927

Epoch 5/24
----------
train Loss: 0.5554 Acc: 0.6222
test Loss: 0.5508 Acc: 0.6316

Epoch 6/24
----------
train Loss: 0.4096 Acc: 0.7235
test Loss: 0.4414 Acc: 0.7067

Epoch 7/24
----------
train Loss: 0.3865 Acc: 0.7395
test Loss: 0.4332 Acc: 0.7117

Epoch 8/24
----------
train Loss: 0.3758 Acc: 0.7456
test Loss: 0.4292 Acc: 0.7147

Epoch 9/24
----------
train Loss: 0.3670 Acc: 0.7510
test Loss: 0.4423 Acc: 0.7081

Epoch 10/24
----------
train Loss: 0.3602 Acc: 0.7555
test Loss: 0.4257 Acc: 0.7176

Epoch 11/24
----------
train Loss: 0.3541 Acc: 0.7593
test Loss: 0.4027 Acc